In [ ]:
# llama attention mechanism

In [4]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Tuple, Optional

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA available:", torch.cuda.is_available())
    print("Number of GPUs:", torch.cuda.device_count())

CUDA available: True
Number of GPUs: 1


In [ ]:
# --- Configuration ---

hidden_size = 128
# Dimension of the embedding vector for each token.
# Example: every token (like "dog") is represented by a vector of 128 numbers.

num_attention_heads = 16
# How many attention heads we use in parallel.
# The hidden_size (128) is split across these heads.
# Each head sees only part of the vector → 128 / 16 = 8 dimensions per head.

num_key_value_heads = 4
# Special trick: Grouped-Query Attention (GQA).
# Instead of creating 16 different Key/Value heads,
# we only create 4 K/V heads (each of size 8) and let multiple Q heads share them.
# → This saves memory and computation while keeping good performance.

head_dim = hidden_size // num_attention_heads
# Size of each head’s Q, K, V vector.
# With hidden_size=128 and 16 heads, each head works in 8 dimensions.

max_position_embeddings = 256
# Maximum sequence length (number of tokens) the model can process at once.
# If a sentence has more than 256 tokens, it must be truncated or split.

rope_theta = 10000.0
# Base frequency for Rotary Position Embeddings (RoPE).
# It controls how positional information is encoded.
# Larger theta = slower change in frequency = smoother positional encoding.

rms_norm_eps = 1e-5
# Tiny constant added inside RMSNorm to avoid division by zero.
# Ensures stability in training and inference.

attention_bias = False
# Whether to add a bias term to the linear layers that produce Q, K, V.
# Usually kept False for efficiency.

attention_dropout = 0.0
# Dropout probability applied to attention weights (to prevent overfitting).
# Often set to 0.0 during inference (disabled).

use_qk_norm = True
# Whether to normalize Q and K vectors (L2 norm) before computing attention scores.
# This keeps dot products more stable and avoids extreme attention weights.
